# Behavioural Cloning Network 
##### this notebook contains code to drive a car autonomously through the Udacity Car Simulator 

In [1]:
import csv 
import cv2 
import numpy as np  
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

#Reading the CSV data file and saving the images for validation data

lines = [] 
with open('./udacity_data/driving_log.csv') as csvfile:
    next(csvfile)
    reader = csv.reader(csvfile)
    for line in reader: 
        lines.append(line)
        
print ('Creating data arrays...')
images = [] 
measurements = [] 
for line in lines: 
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = './udacity_data/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3]) 
    measurements.append(measurement)

X_val = np.array(images)
y_val = np.array(measurements)


print('Finished loading validation data.')

Creating data arrays...
Finished loading validation data.


In [2]:
# Reading the CSV data file and saving the images for train data 

lines = [] 
with open('./train_data/driving_log.csv') as csvfile:
    next(csvfile)
    reader = csv.reader(csvfile)
    for line in reader: 
        lines.append(line)
        
print ('Creating data arrays...')
images = [] 
measurements = [] 
for line in lines: 
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = './train_data/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3]) 
    measurements.append(measurement)


print('Finished loading first set of train data.')

extra_train_sets = 1

for i in range(extra_train_sets):
    lines = [] 
    path = './train_data' + str(i+1) + '/driving_log.csv'
    with open(path) as csvfile:
        next(csvfile)
        reader = csv.reader(csvfile)
        for line in reader: 
            lines.append(line)
    X_data = images 
    y_data = measurements
    print ('Creating data arrays...')
    

    path = './train_data' + str(i+1) + '/IMG/'
    for line in lines: 
        source_path = line[0]
        filename = source_path.split('/')[-1]
        current_path = path + filename
        image = cv2.imread(current_path)
        images.append(image)
        measurement = float(line[3]) 
        measurements.append(measurement)

    print('Finished loading ' + str(i+2) + 'th set of train data.')

print('Done')
X_data = np.array(images)
y_data = np.array(measurements)
print(X_data.shape)

NameError: name 'csv' is not defined

In [10]:
#image preprocessing steps 
import csv 
import cv2 
import numpy as np  
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.data_utils import Sequence
import threading
import multiprocessing 
datagen = ImageDataGenerator(
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


def generate_images(x):
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
    i = 0
    for batch in datagen.flow(x, batch_size=1):
        return np.squeeze(batch)


validation_samples = [] 

with open('./udacity_data/driving_log.csv') as csvfile:
    next(csvfile)
    reader = csv.reader(csvfile)
    for line in reader: 
        validation_samples.append(line)

train_samples = []
with open('./train_data/driving_log.csv') as csvfile:
    next(csvfile)
    reader = csv.reader(csvfile)
    for line in reader: 
        train_samples.append(line)        

with open('./train_data1/driving_log.csv') as csvfile:
    next(csvfile)
    reader = csv.reader(csvfile)
    for line in reader: 
        train_samples.append(line) 

class TrainingSequence(Sequence):
    def __init__(self, x_set, y_set,dir_name, batch_size):
        self.X,self.y = x_set,y_set
        self.batch_size = batch_size
        self.dir_name = dir_name
        
    def __len__(self):
        return len(self.X) 

    def __getitem__(self,idx):
        #print ("Process: " + multiprocessing.current_process().name + "idx is " + str(idx))
        batch_samples = self.X[idx*self.batch_size:(idx+1)*self.batch_size]
        
        images = []
        angles = []
        for batch_sample in batch_samples:
            index = 0 
            rand_i = np.random.randint(low=0,high=2)
            #print(batch_sample[rand_i].split('/')[-1])
            path = './'+ self.dir_name[index] + '/IMG/'+batch_sample[rand_i].split('/')[-1]                 
            image = cv2.imread(path)
            if image is None: 
                index += 1
                path = './'+ self.dir_name[index] +'/IMG/'+batch_sample[rand_i].split('/')[-1] 
                image = cv2.imread(path)
            image = generate_images(np.array(image))
            angle = float(batch_sample[3])
            if rand_i == 1: 
                angle += 0.25 
            elif rand_i == 2: 
                angle -= 0.25
            images.append(image[64:135,:,:])
            angles.append(angle)
        X_train = np.array(images)
        y_train = np.array(angles)
        return X_train, y_train




In [11]:
from keras.models import Sequential 
from keras.layers import Flatten, Dense, Lambda, Conv2D
from keras.backend import tf as ktf 

def NvidiaNet(): 
    
    #resizing & normalization lambda layers
    resize = Lambda(lambda img: ktf.image.resize_images(img, (66, 200)), input_shape=(None, 320, 3))
    normalize = Lambda(lambda x: (x / 255.0) - 0.5)
    
    model = Sequential() 
    model.add(resize) 
    model.add(normalize)
    
    # input: 3@66x200 conv ouput: 24@31x98
    model.add(Conv2D(24, kernel_size=(5,5), strides=(2, 2), padding='valid', activation='relu', kernel_initializer='he_normal', bias_initializer='zeros'))
    
    # conv output: 36@14x47
    model.add(Conv2D(36, kernel_size=(5,5), strides=(2, 2), padding='valid', activation='relu', kernel_initializer='he_normal', bias_initializer='zeros'))          
    
    # conv output: 48@5x22
    model.add(Conv2D(48, kernel_size=(5,5), strides=(2, 2), padding='valid', activation='relu', kernel_initializer='he_normal', bias_initializer='zeros'))    
    
    # conv output: 64@3x20
    model.add(Conv2D(64, kernel_size=(3,3), strides=(1, 1), padding='valid', activation='relu', kernel_initializer='he_normal', bias_initializer='zeros')) 
    
    # conv output: 64@1x18 
    model.add(Conv2D(64, kernel_size=(3,3), strides=(1, 1), padding='valid', activation='relu', kernel_initializer='he_normal', bias_initializer='zeros')) 
    
    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(1164, kernel_initializer='he_normal', bias_initializer='zeros'))
    model.add(Dense(100, kernel_initializer='he_normal', bias_initializer='zeros'))
    model.add(Dense(50, kernel_initializer='he_normal', bias_initializer='zeros')) 
    model.add(Dense(10, kernel_initializer='he_normal', bias_initializer='zeros'))
    model.add(Dense(1, kernel_initializer='he_normal', bias_initializer='zeros'))
    
    model.compile(loss='mse', optimizer='adam',  metrics=['accuracy'])
    
    return model 


def SimpleNet():
    
    input_shape=(160,320,3)
    model = Sequential() 
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=input_shape))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(1)) 

    model.compile(loss='mse', optimizer='adam',  metrics=['accuracy'])
    
    return model

    

In [12]:
import keras
print(keras.__version__)
model = NvidiaNet()
batch_size = 256

#model.fit(X_val, y_val, validation_split=0.2, shuffle=True, nb_epoch=7)
#model.fit(X_data, y_data, batch_size=256, shuffle=True, epochs=25, validation_data=(X_val, y_val))
# compile and train the model using the generator function
var = 0
train_generator = TrainingSequence(train_samples,None, ['train_data', 'train_data1'], batch_size=batch_size)
validation_generator = TrainingSequence(validation_samples,None, ['udacity_data'], batch_size=batch_size)
model.fit_generator(train_generator, validation_data=validation_generator, validation_steps=len(validation_samples)/batch_size , epochs=1, steps_per_epoch=len(train_samples)/batch_size, workers=4, use_multiprocessing=True)
model.save('model.h5')

#./udacity_data/IMG/center_2016_12_01_13_30_48_287.jpg

2.0.6
Epoch 1/1
44/43 [==============================] - 63s - loss: 0.2512 - acc: 0.1973 - val_loss: 0.0338 - val_acc: 0.2675
